DJ Tracklists Scraper
=======================

Get Webpage(s)

In [ ]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import urllib3
import os
import spotipy
import spotipy.util as util
from pprint import pprint
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

Extract data 
<b>TODO: </b>Find a way to bypass cookie problems...

In [ ]:
def get_tracklist(url, folder='.'):
    os.makedirs(folder, exist_ok=True)
    page = requests.get(url, verify=False)
    soup = bs(page.content, "lxml")
    
    tracklist = pd.DataFrame(columns=['Artist(s)', 'Title', 'Release'])
    set_name = soup.select('.set')[0].get_text()
    for div in soup.find_all('td'):
        try:
            artists_raw = div.select('.artist')
            artists = str()
            if(len(artists_raw) == 0):
                continue
            for artist in artists_raw:
                text = artist.get_text()
                artists+=f'{text} '
            # get rid of space at the end    
            artists = artists.strip()

            title = div.select('.track')[0].get_text()
            release = div.select('.release')[0].get_text()

            basic_details = pd.Series([artists, title, release], index=['Artist(s)', 'Title', 'Release'])
            main_artist = artists_raw[0].get_text()
            spotify_details = get_attrs(artist=main_artist+' '+release, track=title)
            # Try removing version if not found
            if spotify_details is False:
                spotify_details = get_attrs(artist=main_artist, track=title)
            # Set details to none instead of false if not found, so track won't get excluded
            if spotify_details is False:
                spotify_details = None
                print(f'Not Found: {title} by {artists}')
            row = pd.concat([basic_details, spotify_details])
            tracklist = tracklist.append(row, ignore_index=True)
        except Exception as e:
            print(e)
            pass
    tracklist.to_csv(f'{folder}/{set_name}.csv')
    
# get_tracklist(url='https://www.djtracklists.com/tracklists/view/future_sound_of_egypt_013/', folder='/tmp')

In [ ]:
client_id='6389b29d73fc4806ba5e812e678854c1'
client_secret='0b4bcd832e694aedad408b5b4a93dd5c'
ccm=util.oauth2.SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp =spotipy.Spotify(client_credentials_manager=ccm)

def get_attrs(artist, track):
    try:
        res = sp.search(q='artist:'+artist+' track:'+track, type="track")
        track_res = res['tracks']['items'][0]
        track_id = track_res['uri']
        deets = sp.audio_features(track_id)
        return pd.Series(deets[0])
    except Exception as e:
        return False
    
#get_attrs(artist='Armin Van Buuren', track="shivers")

TODO
----
+ Convert key signatures to camelot for ml algorithm to use
+ ![Spotify pitch to regular key](./pitch-class-to-key.png)

In [ ]:
def get_camelot(key, mode):
    # index of letter is spotify pitch class - e.g. 0->c, 1->c# etc.
    tones = [ 'c', 'c#', 'd', 'd#', 'e', 'f', 'f#', 'g', 'g#', 'a', 'a#', 'b' ]
    keys = {
        'a0': '8A',
        'a1': '11B',
        'a#0': '3A',
        'a#1': '6B',
        'b0': '10A',
        'b1': '1B',
        'c0': '5A',
        'c1': '8B',
        'c#0': '12A',
        'c#1': '3B',
        'd0': '7A',
        'd1': '10B',
        'd#0': '2A',
        'd#1': '5B',
        'e0': '9A',
        'e1': '12B',
        'f0': '4A',
        'f1': '7B',
        'f#0': '11A',
        'f#1': '2B',
        'g0': '6A',
        'g1': '9B',
        'g#0': '1A',
        'g#1': '4B',
    }
    key_letter = tones[key]
    return keys[key_letter+str(mode)]

In [ ]:
def get_series_tracklists(series_url, folder='.', recursecall=False):
    page = requests.get(series_url, verify=False)
    soup = bs(page.content, "lxml")
    for mix_link in soup.find_all('a', class_='mix'):
        mix_href = mix_link['href']
        print(mix_link['href'])
        get_tracklist(mix_link['href'], folder=folder)
        
    if recursecall == False:
        page_div = soup.find('div', class_='pagination')
        other_pages = page_div.find_all('a', class_='pagenumber', href=True)
        for page in other_pages:
            if page.get_text() != 'Next »»':
                print(page['href'])
                get_series_tracklists(page['href'], folder=folder, recursecall=True)
        
series_urls = { "accelerate": 'https://www.djtracklists.com/series/accelerate_session/',
               "always_alive": 'https://www.djtracklists.com/series/always_alive/',
               "anjunabeats": 'https://www.djtracklists.com/series/anjunabeats_worldwide/',
               "point_blank": 'https://www.djtracklists.com/series/blank_point/',
               "ASOT": 'https://www.djtracklists.com/series/a_state_of_trance/',
               "brija_dot_com": 'https://www.djtracklists.com/series/brija_dot_com_sessions/',
               "captured_radio": 'https://www.djtracklists.com/series/captured_radio/',
               "club_elite_sessions": 'https://www.djtracklists.com/series/club_elite_sessions/',
               "corstens_countdown": 'https://www.djtracklists.com/series/corstens_countdown/',
               "essential_mix": 'https://www.djtracklists.com/series/essential_mix/',
               "FSOE": 'https://www.djtracklists.com/series/future_sound_of_egypt/',
               "GDJB": 'https://www.djtracklists.com/series/global_dj_broadcast/',
               "jaytech_music": 'https://www.djtracklists.com/series/jaytech_music/',
               "magic_island": 'https://www.djtracklists.com/series/magic_island-music_for_balearic_people/',
               "TATW": 'https://www.djtracklists.com/series/trance_around_the_world/',
               "Transitions": 'https://www.djtracklists.com/series/transitions/'
              }
               
for key in series_urls.keys():
    get_series_tracklists(series_urls[key], folder=key)